## **FDS project, winter semester 2023**

#### Tommaso Leonardi, Arianna Paolini, Stefano Saravalle, Paolo Cursi, Pietro Signorino
<leonardi.1914546@studenti.uniroma1.it>, <paolini.1943164@studenti.uniroma1.it>, <saravalle.1948684@studenti.uniroma1.it>, <paoloc1999@gmail.com>, <signorino.2149741@studenti.uniroma1.it>

# **Student Performance Analysis & Prediction**

In [19]:
#import libraries
import pandas as pd

### **Data Preprocessing**

In [20]:
#loading the dataset tables
courses = pd.read_csv('./data/courses.csv')  #22 rows for courses (modules) and their presentations
assess = pd.read_csv('./data/assessments.csv')  #206 rows of assessments for module-presentations (including the final exam)
results = pd.read_csv('./data/studentAssessment.csv') #173,912 rows for the scores obtained by students in the asssesments
studs = pd.read_csv('./data/studentInfo.csv') #32,593 rows for demographic information on students and their results in module-presentations
registr = pd.read_csv('./data/studentRegistration.csv') #32,593 rows for student registration/unregistration on module-presentations
vle = pd.read_csv('./data/studentVle.csv') #10,655,280 rows for daily student interactions with online resources for a module-presentation
materials = pd.read_csv('./data/vle.csv') #6,364 rows for the materials available on the Virtual Learning Environment

The _Open University Learning Analytics dataset_ that we are considering has the following structure: 

 <img src="https://analyse.kmi.open.ac.uk/resources/images/model.png" alt="dataset structure" style="height: 500px; width:500px;"/>


(https://analyse.kmi.open.ac.uk/open_dataset)



In [76]:
#DONE: definire i task -> regressione/classificazione dello score per ogni assessment
#DONE: trasformare valori categorici in numeri
#TODO: normalizzare/pulire i dati
#TODO: mostrare la distribuzione dei dati con grafici
#TODO: fare split tra training e test set (considerare cross validation)


Since our goal is to predict the score of each student in any assessment belonging to a specific module presentation, we consider the demographic information about students (from the table *studentInfo*) and their accessess to online resources in the Virtual Learning Environment (VLE) for each course (from the tables *studentVle* and *vle*) as features for our models. 

We also take in account the assessment type and weigth (from the table *assessments*) and the time the student spent before submitting it (from the table *studentAssessment*). 

The target value to predict is the score from the *studentAssessment* table, which ranges from 0 to 100.

In [21]:
#dropping some features
studs = studs.drop("final_result", axis=1)
registr = registr.drop("date_unregistration", axis=1)
materials = materials.drop(["week_from", "week_to"], axis=1)
vle = vle.drop("date", axis=1)
results = results.drop("is_banked", axis=1)

#match info about a student and his date of registration to a module presentation
studs = studs.merge(registr, how="inner", on=["code_module", "code_presentation","id_student"])

#match a student's interactions with an online resource with the type of the resource
vle = vle.merge(materials, how="inner", on=["code_module", "code_presentation","id_site"] )
vle = vle.drop("id_site", axis=1)

#group the interactions by resource type and consider the total sum of clicks
vle = vle.groupby(["code_module", "code_presentation", "id_student", "activity_type"]).sum().reset_index() #TO FIX

#match student's information with his interactions on the VLE for a specific module presentation
studs = studs.merge(vle, how="inner", on=["code_module", "code_presentation", "id_student"])

#match assessments with students scores
assess = assess.merge(results, how="inner", on="id_assessment")

#substitute date in assessment and date_submitted in results with their difference (to be considered as a time interval)
assess["submission_interval"] = assess["date"] - assess["date_submitted"] #"date" is the deadline for the assessment
assess = assess.drop(["date","date_submitted"], axis=1)

#match students with their results
df = studs.merge(assess, how="inner", on=["code_module", "code_presentation","id_student"])

#remove ids from the features
df = df.drop(["code_module","code_presentation","id_student","id_assessment"], axis=1)

In [22]:
df

,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,date_registration,activity_type,sum_click,assessment_type,weight,score,submission_interval
0,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,-159.0,forumng,193,TMA,10.0,78.0,1.0
1,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,-159.0,forumng,193,TMA,20.0,85.0,1.0
2,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,-159.0,forumng,193,TMA,20.0,80.0,2.0
3,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,-159.0,forumng,193,TMA,20.0,85.0,2.0
4,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,-159.0,forumng,193,TMA,30.0,82.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1603453,F,Yorkshire Region,HE Qualification,50-60%,35-55,0,30,N,-28.0,subpage,27,CMA,0.0,100.0,57.0
1603454,F,Yorkshire Region,HE Qualification,50-60%,35-55,0,30,N,-28.0,subpage,27,CMA,0.0,100.0,23.0
1603455,F,Yorkshire Region,HE Qualification,50-60%,35-55,0,30,N,-28.0,subpage,27,TMA,0.0,80.0,24.0
1603456,F,Yorkshire Region,HE Qualification,50-60%,35-55,0,30,N,-28.0,subpage,27,TMA,0.0,80.0,10.0


Function to parse from string values to integers. Just change the list to_be_parsed if you want to add or not consider a certain column.

In [24]:
to_be_parsed = ["gender", "region", "highest_education", "imd_band", "age_band", "disability", "activity_type", "assessment_type"]

for column_name in to_be_parsed:

    values = set(df[column_name].tolist())
    print(f"Values in {column_name} column: {values}")

    mapping = {x:y for y,x in enumerate(values)}
    print(f"Mapping from string values to numerical using the following dictionary: {mapping}")

    df[column_name] = df[column_name].map(mapping)

    print("\n==================================================\n")


Values in gender column: {'F', 'M'}
Mapping from string values to numerical using the following dictionary: {'F': 0, 'M': 1}


Values in region column: {'Wales', 'East Midlands Region', 'Yorkshire Region', 'North Region', 'East Anglian Region', 'Scotland', 'London Region', 'North Western Region', 'Ireland', 'South West Region', 'South Region', 'West Midlands Region', 'South East Region'}
Mapping from string values to numerical using the following dictionary: {'Wales': 0, 'East Midlands Region': 1, 'Yorkshire Region': 2, 'North Region': 3, 'East Anglian Region': 4, 'Scotland': 5, 'London Region': 6, 'North Western Region': 7, 'Ireland': 8, 'South West Region': 9, 'South Region': 10, 'West Midlands Region': 11, 'South East Region': 12}


Values in highest_education column: {'Lower Than A Level', 'No Formal quals', 'A Level or Equivalent', 'HE Qualification', 'Post Graduate Qualification'}
Mapping from string values to numerical using the following dictionary: {'Lower Than A Level': 0, 'N

In [26]:
df

,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,date_registration,activity_type,sum_click,assessment_type,weight,score,submission_interval
0,1,4,3,2,1,0,240,1,-159.0,0,193,2,10.0,78.0,1.0
1,1,4,3,2,1,0,240,1,-159.0,0,193,2,20.0,85.0,1.0
2,1,4,3,2,1,0,240,1,-159.0,0,193,2,20.0,80.0,2.0
3,1,4,3,2,1,0,240,1,-159.0,0,193,2,20.0,85.0,2.0
4,1,4,3,2,1,0,240,1,-159.0,0,193,2,30.0,82.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1603453,0,2,3,7,2,0,30,1,-28.0,18,27,0,0.0,100.0,57.0
1603454,0,2,3,7,2,0,30,1,-28.0,18,27,0,0.0,100.0,23.0
1603455,0,2,3,7,2,0,30,1,-28.0,18,27,2,0.0,80.0,24.0
1603456,0,2,3,7,2,0,30,1,-28.0,18,27,2,0.0,80.0,10.0
